Pour charger et préparer vos fichiers `badqueries.txt` et `goodqueries.txt` pour l'entraînement d'un modèle de machine learning avec Python et scikit-learn, vous pouvez suivre les étapes ci-dessous. Nous allons lire les fichiers, les étiqueter, les combiner en un seul dataframe, puis les préparer pour le processus d'apprentissage.

### Étape 1 : Charger les données

Assurez-vous que Python peut accéder aux fichiers où ils sont stockés sur votre système. Si nécessaire, ajustez le chemin d'accès aux fichiers.


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer


In [4]:


# Charger les données
def load_data(filepath, label):
    with open(filepath, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        data = [(line.strip(), label) for line in lines if line.strip()]
    return data

# Emplacements des fichiers
bad_queries_path = 'badqueries.txt'
good_queries_path = 'goodqueries.txt'

# Charger et étiqueter les données
bad_data = load_data(bad_queries_path, 1)  # Étiquette 1 pour les mauvaises requêtes
good_data = load_data(good_queries_path, 0)  # Étiquette 0 pour les bonnes requêtes

# Combinez les données dans un DataFrame
all_data = pd.DataFrame(bad_data + good_data, columns=['query', 'label'])


### Petite exploration des données

In [5]:
print(all_data.columns)
print(all_data.shape)
print(all_data.head())

Index(['query', 'label'], dtype='object')
(1342596, 2)
                                               query  label
0                 /top.php?stuff='uname >q36497765 #      1
1  /h21y8w52.nsf?<script>cross_site_scripting.nas...      1
2  /ca000001.pl?action=showcart&hop=\"><script>al...      1
3  /scripts/edit_image.php?dn=1&userfile=/etc/pas...      1
4                                /javascript/mta.exe      1




### Étape 2 : Préparation des données

Nous diviserons les données en ensembles d'entraînement et de test, puis appliquerons la vectorisation TF-IDF pour convertir les requêtes textuelles en vecteurs numériques.

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Séparer les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(all_data['query'], all_data['label'], test_size=0.2, random_state=42)



In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re

def custom_tokenizer(text):
    # Utilisez une expression régulière pour considérer les mots et certains symboles spéciaux
    token_pattern = re.compile(r"(?u)\b\w\w+\b|\/|<|>|script|alert|http|\.exe|\.jpg|=")
    return token_pattern.findall(text)

vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer, lowercase=False, strip_accents=None)

# Exemple d'utilisation
example_data = [
    "/examples/jsp/cal/search.php?allwords=<br><script>foo</script>&cid=0&title=1&desc=1",
    "/moodle/filter/tex/texed.php?formdata=foo&pathname=foo\"+||+echo+db+4d+5a+50+00+02+00+00+00+04+00+0f+00+ff+ff+00+00+b8+00+00+00+00+00+00+00+40++>>esbq"
]

X = vectorizer.fit_transform(example_data)
print(vectorizer.get_feature_names_out())  # Affiche les termes du vocabulaire


['/' '00' '02' '04' '0f' '40' '4d' '50' '5a' '<' '=' '>' 'allwords' 'b8'
 'br' 'cal' 'cid' 'db' 'desc' 'echo' 'esbq' 'examples' 'ff' 'filter' 'foo'
 'formdata' 'jsp' 'moodle' 'pathname' 'php' 'script' 'search' 'tex'
 'texed' 'title']


/opt/mamba/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [8]:
# Vectorisation TF-IDF

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

Pour comprendre à quoi ressemble `X_train_tfidf` après vectorisation avec TF-IDF, vous pouvez visualiser quelques aspects de cette matrice. `X_train_tfidf` est une matrice sparse (creuse) de `scikit-learn`, ce qui signifie qu'elle stocke uniquement les emplacements et les valeurs des éléments non nuls pour économiser de l'espace mémoire.

### 1. Propriétés de base de la matrice


In [9]:
print("Shape of X_train_tfidf:", X_train_tfidf.shape)  # dimensions de la matrice
print("Type of X_train_tfidf:", type(X_train_tfidf))   # type de l'objet (sparse matrix)

Shape of X_train_tfidf: (1074076, 755132)
Type of X_train_tfidf: <class 'scipy.sparse._csr.csr_matrix'>


### 2. Vérifier le nombre d'éléments non nuls

In [10]:
print("Number of non-zero elements:", X_train_tfidf.nnz)
print("Density of the matrix:", X_train_tfidf.nnz / float(X_train_tfidf.shape[0] * X_train_tfidf.shape[1]))

Number of non-zero elements: 3318992
Density of the matrix: 4.0921196018022575e-06


### 3. Affichage des termes du vocabulaire

In [11]:
feature_names = vectorizer.get_feature_names_out()
print("Some feature names:", feature_names[:20])  # affiche les 20 premiers termes du vocabulaire

Some feature names: ['.exe' '.jpg' '/' '00' '000' '0000' '000000' '0000000' '00000000'
 '000000000' '00000000000000000020195001' '00000000000000097vascript'
 '000000000001' '0000000001' '0000000001_000000000000000051240'
 '0000000001_000000000000000122701' '000000000663' '000000000667'
 '000000000699' '000000000788']


Votre mise en œuvre du `TfidfVectorizer` avec un tokenizer personnalisé semble bien adaptée pour gérer les requêtes potentiellement malveillantes que vous avez mentionnées. Vous avez bien configuré les étapes pour diviser vos données et appliquer la vectorisation TF-IDF, ce qui est crucial pour la suite de l'analyse avec un modèle de machine learning.

### Points à considérer :
1. **Validation du Tokenizer :** Assurez-vous que le tokenizer personnalisé extrait bien les termes pertinents de vos données d'entraînement et de test. Les termes imprimés depuis `example_data` donnent une bonne idée des tokens que votre vectorisateur considère.

2. **Entraînement et transformation :** Vous avez utilisé `fit_transform` sur `X_train` et `transform` sur `X_test`, ce qui est la méthode correcte pour éviter la fuite de données de l'ensemble de test dans le modèle de vectorisation.

3. **Gestion des caractères et motifs spéciaux :** Votre expression régulière semble bien conçue pour capturer à la fois les mots standards et les motifs spécifiques aux scripts malveillants, ce qui est essentiel pour ce type d'analyse.

### Étapes suivantes :

Maintenant que vos données sont prêtes, l'étape suivante consiste à choisir et entraîner un modèle de machine learning pour classifier les requêtes comme bonnes ou mauvaises. Voici quelques options de modèles que vous pourriez envisager :

- **Régression logistique :** Un modèle simple mais efficace pour les tâches de classification binaire.
- **Machines à vecteurs de support (SVM) :** Très efficace pour les espaces de grande dimension comme ceux créés par TF-IDF.
- **Forêts aléatoires :** Bon pour gérer des données avec beaucoup de variance et pour capturer des non-linéarités sans trop de tuning nécessaire.
- **Réseaux de neurones :** Si vous avez suffisamment de données et de ressources de calcul, un réseau de neurones pourrait capter des interactions complexes entre les mots.

Chaque modèle a ses avantages et ses inconvénients, et le choix peut dépendre de la taille de vos données, de la précision requise, du temps de calcul disponible, et de votre familiarité avec les modèles.

Si vous souhaitez des conseils sur l'entraînement d'un modèle spécifique ou sur l'évaluation de la performance du modèle, n'hésitez pas à demander !

### Régression logistique

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Création du modèle de régression logistique
logreg = LogisticRegression(random_state=42)

# Entraînement du modèle avec les données d'entraînement
logreg.fit(X_train_tfidf, y_train)

# Prédiction des étiquettes sur l'ensemble de test
y_pred = logreg.predict(X_test_tfidf)

# Calcul et affichage des métriques de performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9944212721584984
Confusion Matrix:
 [[258575    446]
 [  1052   8447]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    259021
           1       0.95      0.89      0.92      9499

    accuracy                           0.99    268520
   macro avg       0.97      0.94      0.96    268520
weighted avg       0.99      0.99      0.99    268520



Votre modèle de régression logistique semble obtenir d'excellents résultats globalement, comme le montre la très haute précision globale (accuracy) de 99.44%. Cependant, il est toujours utile de regarder un peu plus en détail chaque métrique pour bien comprendre les performances du modèle, surtout dans le contexte de la classification de requêtes potentiellement malveillantes.

### Analyse des Résultats

1. **Précision globale (Accuracy) :** À 99.44%, la plupart des prédictions sont correctes. C'est une bonne indication que le modèle fonctionne bien en général.

2. **Matrice de confusion :** 
   - **Vrais positifs pour la classe 0 (bonnes requêtes)** : 258575
   - **Faux positifs pour la classe 0** : 446
   - **Faux négatifs pour la classe 0** : 1052
   - **Vrais positifs pour la classe 1 (mauvaises requêtes)** : 8447

   Les faibles nombres de faux positifs et de faux négatifs suggèrent que le modèle est à la fois précis et sensible. Cependant, le nombre de faux négatifs est supérieur aux faux positifs, ce qui indique une légère faiblesse dans la capture de toutes les mauvaises requêtes (moins de sensibilité).

3. **Rapport de classification :**
   - **Précision pour la classe 0** : Pratiquement parfaite à 1.00.
   - **Précision pour la classe 1** : À 0.95, c'est très bon, mais cela indique aussi que certains faux positifs sont présents.
   - **Rappel pour la classe 0** : Aussi pratiquement parfait à 1.00.
   - **Rappel pour la classe 1** : À 0.89, ce qui indique que le modèle manque environ 11% des mauvaises requêtes réelles.
   - **F1-score pour la classe 1** : À 0.92, ce score équilibré entre précision et rappel est assez bon, mais peut être amélioré, surtout si l'identification précise des mauvaises requêtes est critique.

### Suggestions pour l'amélioration

- **Ajustement des hyperparamètres :** Vous pouvez tenter d'ajuster le paramètre de régularisation `C` de la régression logistique pour voir si une valeur plus faible (plus de régularisation) ou plus élevée (moins de régularisation) améliore les performances.

- **Équilibrage des classes :** Si votre jeu de données est très déséquilibré, envisagez des techniques comme le rééchantillonnage (oversampling de la classe minoritaire ou undersampling de la classe majoritaire) ou l'utilisation de poids de classe dans le modèle de régression logistique pour donner plus de poids à la classe minoritaire pendant l'entraînement.

- **Validation croisée :** Utiliser une validation croisée pour évaluer la robustesse du modèle sur différents sous-ensembles de vos données peut aider à garantir que les performances ne sont pas spécifiques à une division particulière des données.

- **Essayer d'autres modèles :** Parfois, un modèle différent peut capturer les dynamiques des données plus efficacement. Les machines à vecteurs de support (SVM), les forêts aléatoires ou même un modèle de réseaux de neurones pourraient être testés pour comparer les performances.


### Prédiction avec des données issues de Splunk

In [13]:
# Prédire avec une nouvelle query
new_query = ["web4gfo.preprod.insee.fr:443/fr/graphique/affaires"]
new_query_tfidf = vectorizer.transform(new_query)
prediction = logreg.predict(new_query_tfidf)
prediction_proba = logreg.predict_proba(new_query_tfidf)

print(f"Query: {new_query[0]}")
print(f"Prediction: {'Bad Query' if prediction[0] == 1 else 'Good Query'}")
print(f"Prediction Probability: {prediction_proba[0]}")


Query: web4gfo.preprod.insee.fr:443/fr/graphique/affaires
Prediction: Good Query
Prediction Probability: [0.99601602 0.00398398]


Le résultat de la prédiction peut être interprété comme suit :

### Query:
`web4gfo.preprod.insee.fr:443/fr/graphique/affaires`

Cette est la requête que nous avons soumise pour la prédiction.

### Prediction:
`Good Query`

Le modèle a prédit que cette requête est une "Good Query" (bonne requête). Cela signifie que, selon le modèle, cette requête n'est pas malveillante ou problématique.

### Prediction Probability:
`[0.99601602 0.00398398]`

Ce tableau de probabilités fournit des informations supplémentaires sur la confiance du modèle dans sa prédiction :
- **0.99601602** (99.60%) : C'est la probabilité que la requête soit une bonne requête.
- **0.00398398** (0.40%) : C'est la probabilité que la requête soit une mauvaise requête.

### Interprétation:

Le modèle est très confiant (99.60%) que la requête `web4gfo.preprod.insee.fr:443/fr/graphique/affaires` est une bonne requête. La très faible probabilité de 0.40% que cette requête soit une mauvaise requête indique que le modèle a peu de doutes quant à sa classification.

En résumé, selon le modèle de régression logistique, cette requête est presque certainement bénigne et non malveillante.


1. **Chargement et préparation des données**:
   - Les fichiers `badqueries.txt` et `goodqueries.txt` sont chargés et étiquetés.
   - Les données sont combinées dans un DataFrame et divisées en ensembles d'entraînement et de test.

2. **Tokenisation personnalisée**:
   - Une fonction de tokenisation personnalisée est définie pour extraire des tokens spécifiques des requêtes.

3. **Vectorisation TF-IDF**:
   - Les données sont transformées en vecteurs TF-IDF en utilisant la fonction de tokenisation personnalisée.

4. **Modèle de régression logistique**:
   - Un modèle de régression logistique est formé sur les données vectorisées.

5. **Prédiction avec une nouvelle query**:
   - La nouvelle query est transformée en vecteur TF-IDF et utilisée pour prédire si elle est une "bonne" ou "mauvaise" requête.

6. **Affichage des résultats**:
   - La prédiction et les probabilités associées sont affichées.
   - Les métriques de performance sur les données de test sont également affichées pour évaluer le modèle.



Pour améliorer la précision d'un modèle de régression logistique ou tout autre modèle de classification, plusieurs stratégies peuvent être mises en œuvre. Voici quelques-unes des méthodes courantes :

### 1. **Collecte et Préparation des Données**
- **Augmenter la Taille du Jeu de Données** : Plus de données peuvent aider le modèle à mieux généraliser.
- **Équilibrer le Jeu de Données** : Assurez-vous que les classes sont équilibrées pour éviter le biais vers la classe majoritaire.
- **Nettoyage des Données** : Supprimez les valeurs aberrantes et corrigez les erreurs dans les données.

### 2. **Feature Engineering**
- **Créer de Nouvelles Caractéristiques** : Identifier et ajouter de nouvelles caractéristiques pertinentes à partir des données existantes.
- **Sélection de Caractéristiques** : Utilisez des techniques comme la sélection de caractéristiques (Feature Selection) pour garder seulement les plus pertinentes.
- **Transformation des Caractéristiques** : Appliquer des transformations logarithmiques ou de normalisation aux caractéristiques numériques.

### 3. **Optimisation du Modèle**
- **Ajustement des Hyperparamètres** : Utilisez des techniques comme la recherche en grille (Grid Search) ou la recherche aléatoire (Random Search) pour trouver les meilleurs hyperparamètres.
- **Validation Croisée** : Utilisez la validation croisée pour évaluer la performance du modèle de manière plus robuste.

### 4. **Enrichissement des Caractéristiques Textuelles**
- **Utiliser des Modèles de Langage Avancés** : Envisagez d'utiliser des modèles comme BERT ou TF-IDF pondéré.
- **Améliorer la Tokenisation** : Utilisez des méthodes de tokenisation plus sophistiquées qui prennent en compte le contexte des mots.

### 5. **Ensembles de Modèles**
- **Ensemble Learning** : Combinez les prédictions de plusieurs modèles pour améliorer la robustesse et la précision (par exemple, en utilisant Bagging, Boosting ou Stacking).

### 6. **Algorithmes Alternatifs**
- **Essayer d'Autres Algorithmes** : Envisagez d'utiliser d'autres modèles comme les forêts aléatoires, les SVMs, les réseaux neuronaux, etc.

### Exemple d'Amélioration

Voici un exemple de code illustrant certaines de ces améliorations, en particulier l'optimisation des hyperparamètres et l'utilisation de validation croisée :


### Explications du Code :

1. **Pipeline** : Utilisation de `Pipeline` pour combiner le vecteur TF-IDF et le modèle de régression logistique.
2. **Recherche en Grille** : Définition d'une grille de paramètres à tester pour trouver les meilleurs hyperparamètres.
3. **Validation Croisée** : Utilisation de la validation croisée pour évaluer la performance du modèle avec différentes configurations.
4. **Prédiction** : Utilisation du meilleur modèle trouvé pour prédire la classe et les probabilités pour une nouvelle requête.



In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import re

In [15]:


# Charger les données
def load_data(filepath, label):
    with open(filepath, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        data = [(line.strip(), label) for line in lines if line.strip()]
    return data

# Emplacements des fichiers
bad_queries_path = 'badqueries.txt'
good_queries_path = 'goodqueries.txt'

# Charger et étiqueter les données
bad_data = load_data(bad_queries_path, 1)  # Étiquette 1 pour les mauvaises requêtes
good_data = load_data(good_queries_path, 0)  # Étiquette 0 pour les bonnes requêtes

# Combinez les données dans un DataFrame
all_data = pd.DataFrame(bad_data + good_data, columns=['query', 'label'])

# Séparer les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(all_data['query'], all_data['label'], test_size=0.2, random_state=42)

# Définir une fonction de tokenisation personnalisée
def custom_tokenizer(text):
    token_pattern = re.compile(r"(?u)\b\w\w+\b|\/|<|>|script|alert|http|\.exe|\.jpg|=")
    return token_pattern.findall(text)

# Pipeline avec Vectorizer et Logistic Regression
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(tokenizer=custom_tokenizer, lowercase=False, strip_accents=None)),
    ('classifier', LogisticRegression())
])

# Paramètres à tester lors de la recherche en grille
param_grid = {
    'vectorizer__max_df': [0.75, 0.85, 1.0],
    'vectorizer__min_df': [1, 2, 3],
    'classifier__C': [0.1, 1, 10],
    'classifier__solver': ['liblinear', 'saga']
}

# Recherche en grille avec validation croisée
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)




/opt/mamba/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/mamba/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/mamba/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/mamba/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/mamba/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/mamba/lib/python3.12/site-packages/sklearn/feature_extr

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vectorizer',
                                        TfidfVectorizer(lowercase=False,
                                                        tokenizer=<function custom_tokenizer at 0x7f366dd240e0>)),
                                       ('classifier', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'classifier__C': [0.1, 1, 10],
                         'classifier__solver': ['liblinear', 'saga'],
                         'vectorizer__max_df': [0.75, 0.85, 1.0],
                         'vectorizer__min_df': [1, 2, 3]},
             scoring='accuracy')

In [16]:
# Afficher les meilleurs paramètres et la meilleure précision
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validation Accuracy: {grid_search.best_score_}")

# Prédiction avec les meilleurs paramètres
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Afficher les métriques de performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Prédire avec une nouvelle query
new_query = ["web4gfo.preprod.insee.fr:443/fr/graphique/affaires"]
new_query_tfidf = best_model.named_steps['vectorizer'].transform(new_query)
prediction = best_model.named_steps['classifier'].predict(new_query_tfidf)
prediction_proba = best_model.named_steps['classifier'].predict_proba(new_query_tfidf)

print(f"Query: {new_query[0]}")
print(f"Prediction: {'Bad Query' if prediction[0] == 1 else 'Good Query'}")
print(f"Prediction Probability: {prediction_proba[0]}")

Best Parameters: {'classifier__C': 10, 'classifier__solver': 'liblinear', 'vectorizer__max_df': 1.0, 'vectorizer__min_df': 1}
Best Cross-Validation Accuracy: 0.9970923845213896


Accuracy: 0.9972553254878593
Confusion Matrix:
 [[258796    225]
 [   512   8987]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    259021
           1       0.98      0.95      0.96      9499

    accuracy                           1.00    268520
   macro avg       0.99      0.97      0.98    268520
weighted avg       1.00      1.00      1.00    268520

Query: web4gfo.preprod.insee.fr:443/fr/graphique/affaires
Prediction: Good Query
Prediction Probability: [9.99358416e-01 6.41583778e-04]


### Explication en Détail

#### Pipeline avec Vectorizer et Logistic Regression

Un pipeline en `scikit-learn` permet d'enchaîner plusieurs étapes de traitement des données et d'entraînement du modèle en un seul objet. Cela simplifie grandement le processus de transformation des données et d'entraînement du modèle, et assure que toutes les étapes sont appliquées de manière cohérente sur les ensembles d'entraînement et de test.

```python
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(tokenizer=custom_tokenizer, lowercase=False, strip_accents=None)),
    ('classifier', LogisticRegression())
])
```

1. **Vectorizer** : La première étape du pipeline est le `TfidfVectorizer`. Ce transformateur convertit les requêtes textuelles en une matrice de caractéristiques en utilisant le TF-IDF (Term Frequency-Inverse Document Frequency). Il utilise une fonction de tokenisation personnalisée définie par `custom_tokenizer`.

2. **Classifier** : La seconde étape du pipeline est le modèle de régression logistique (`LogisticRegression`). Ce modèle est utilisé pour la classification des requêtes en bonnes ou mauvaises.

#### Paramètres à Tester lors de la Recherche en Grille

La recherche en grille (`GridSearchCV`) permet de tester plusieurs combinaisons d'hyperparamètres pour trouver la meilleure configuration pour le modèle.

```python
param_grid = {
    'vectorizer__max_df': [0.75, 0.85, 1.0],
    'vectorizer__min_df': [1, 2, 3],
    'classifier__C': [0.1, 1, 10],
    'classifier__solver': ['liblinear', 'saga']
}
```

1. **vectorizer__max_df** : Paramètre du `TfidfVectorizer` qui ignore les termes ayant une fréquence de document strictement supérieure au seuil donné (exprimé en proportion du nombre total de documents). Les valeurs testées sont 0.75, 0.85, et 1.0.

2. **vectorizer__min_df** : Paramètre du `TfidfVectorizer` qui ignore les termes ayant une fréquence de document strictement inférieure au seuil donné. Les valeurs testées sont 1, 2, et 3.

3. **classifier__C** : Paramètre de régularisation du modèle de régression logistique. Les valeurs testées sont 0.1, 1, et 10. Un plus grand C signifie une régularisation moindre.

4. **classifier__solver** : Algorithme utilisé pour optimiser le modèle de régression logistique. Les valeurs testées sont `liblinear` et `saga`.

#### Recherche en Grille avec Validation Croisée

La validation croisée est une technique pour évaluer la performance d'un modèle de manière plus robuste en le testant sur plusieurs sous-ensembles des données.

```python
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)
```

1. **GridSearchCV** : Recherche les meilleures combinaisons d'hyperparamètres en utilisant la validation croisée (ici, 5 fois, `cv=5`).
2. **scoring='accuracy'** : La mesure de performance utilisée pour évaluer les modèles est la précision.
3. **n_jobs=-1** : Utilise tous les processeurs disponibles pour accélérer la recherche en grille.
4. **fit(X_train, y_train)** : Entraîne le pipeline avec les différentes combinaisons d'hyperparamètres sur les données d'entraînement.

#### Résultats

```python
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validation Accuracy: {grid_search.best_score_}")

# Prédiction avec les meilleurs paramètres
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
```

##### Best Parameters
```plaintext
Best Parameters: {'classifier__C': 10, 'classifier__solver': 'liblinear', 'vectorizer__max_df': 1.0, 'vectorizer__min_df': 1}
```

Les meilleurs hyperparamètres trouvés par la recherche en grille sont :
- **classifier__C** : 10 (Faible régularisation, ce qui permet au modèle de mieux s'ajuster aux données)
- **classifier__solver** : `liblinear` (Un solver approprié pour des petits datasets)
- **vectorizer__max_df** : 1.0 (Utilise tous les termes)
- **vectorizer__min_df** : 1 (Ignore les termes apparaissant dans moins d'un document)

##### Best Cross-Validation Accuracy
```plaintext
Best Cross-Validation Accuracy: 0.9970923845213896
```

La meilleure précision moyenne obtenue lors de la validation croisée est de 99.71%.

##### Accuracy
```plaintext
Accuracy: 0.9972553254878593
```

La précision sur le jeu de données de test est de 99.73%, ce qui signifie que le modèle classifie correctement environ 99.73% des requêtes.

##### Confusion Matrix
```plaintext
Confusion Matrix:
 [[258796    225]
 [   512   8987]]
```

La matrice de confusion montre :
- **258796** vrais négatifs (bonnes requêtes correctement classées)
- **225** faux positifs (bonnes requêtes incorrectement classées comme mauvaises)
- **512** faux négatifs (mauvaises requêtes incorrectement classées comme bonnes)
- **8987** vrais positifs (mauvaises requêtes correctement classées)

##### Classification Report
```plaintext
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    259021
           1       0.98      0.95      0.96      9499

    accuracy                           1.00    268520
   macro avg       0.99      0.97      0.98    268520
weighted avg       1.00      1.00      1.00    268520
```

- **Classe 0 (bonnes requêtes)** :
  - **Précision** : 100% (presque toutes les bonnes requêtes sont correctement identifiées)
  - **Rappel** : 100% (presque toutes les bonnes requêtes sont trouvées)
  - **F1-score** : 100% (combinaison de précision et de rappel)
  
- **Classe 1 (mauvaises requêtes)** :
  - **Précision** : 98% (98% des requêtes identifiées comme mauvaises sont réellement mauvaises)
  - **Rappel** : 95% (95% des mauvaises requêtes sont correctement identifiées)
  - **F1-score** : 96% (bonne balance entre précision et rappel)

Le modèle est très performant, avec une haute précision et rappel pour les deux classes, montrant qu'il est capable de bien distinguer entre les bonnes et mauvaises requêtes.

In [26]:
import joblib
import dill

def custom_tokenizer(text):
    token_pattern = re.compile(r"(?u)\b\w\w+\b|\/|<|>|script|alert|http|\.exe|\.jpg|=")
    return token_pattern.findall(text)

# Exporter le meilleur modèle
best_model = grid_search.best_estimator_
dill_file = "best_logistic_regression_model.pkl"
with open(dill_file, 'wb') as file:
    dill.dump(best_model, file)

print(f"Meilleur modèle sauvegardé sous le nom: {dill_file}")

Meilleur modèle sauvegardé sous le nom: best_logistic_regression_model.pkl


### Application aux données Splunk

In [18]:
import json
import pandas as pd
import re

# Path to the JSON file
file_path = '/home/onyxia/work/ml-dl-cyber/Splunk/1716794372_291602.json'

# Read the file line by line and parse each JSON object
data = []
with open(file_path, 'r') as file:
    for line in file:
        data.append(json.loads(line))

# Extract the 'result' part of each JSON object and create a dataset
results = [entry['result'] for entry in data]

# Convert the list of results to a DataFrame
results_df = pd.DataFrame(results)

# Define a function to extract variables from the _raw column
def extract_variables(raw_string):
    # Define the regular expression pattern with named groups
    pattern = re.compile(
        # Match timestamp (e.g., "May 24 14:33:29")
        r'(?P<timestamp>[A-Za-z]+\s+\d+\s+\d+:\d+:\d+)\s+'
        # Match host IP (e.g., "172.22.7.248")
        r'(?P<host_ip>\d+\.\d+\.\d+\.\d+)\s+'
        # Match any alphanumeric characters, dashes, or dots (e.g., "PD-HLB01-ID261.dc2.sia.priv")
        r'[A-Za-z0-9\-\.]+\s+'
        # Match the info level (e.g., "info")
        r'(?P<info>[a-z]+)\s+'
        # Match tmm with digits in brackets (e.g., "tmm[20235]")
        r'tmm\[\d+\]:\s+'
        # Match the rule description
        r'Rule\s+/Common/Irule-Syslog-splunk_v2\s+<HTTP_RESPONSE>:\s+'
        # Match src_ip (e.g., "src_ip="10.92.115.117%5"")
        r'src_ip="(?P<src_ip>[^"]+)",'
        # Match vip (e.g., "vip="10.210.248.11%5"")
        r'vip="(?P<vip>[^"]+)",'
        # Match http_method (e.g., "http_method="POST"")
        r'http_method="(?P<http_method>[^"]+)",'
        # Match http_host (e.g., "http_host="web4gfo.preprod.insee.fr:443"")
        r'http_host="(?P<http_host>[^"]+)",'
        # Match http_uri (e.g., "http_uri="/fr/graphique/affaires"")
        r'http_uri="(?P<http_uri>[^"]+)",'
        # Match http_url (e.g., "http_url="web4gfo.preprod.insee.fr:443/fr/graphique/affaires"")
        r'http_url="(?P<http_url>[^"]+)",'
        # Match http_version (e.g., "http_version="1.1"")
        r'http_version="(?P<http_version>[^"]+)",'
        # Match http_user_agent (e.g., "http_user_agent="Mozilla/5.0 ..."")
        r'http_user_agent="(?P<http_user_agent>[^"]+)",'
        # Match http_content_type (e.g., "http_content_type="application/x-www-form-urlencoded; charset=UTF-8"")
        r'http_content_type="(?P<http_content_type>[^"]+)",'
        # Match http_referrer (e.g., "http_referrer="https://web4gfo.preprod.insee.fr/fr/accueil"")
        r'http_referrer="(?P<http_referrer>[^"]+)",'
        # Match req_start_time (e.g., "req_start_time="2024/05/24 14:33:29"")
        r'req_start_time="(?P<req_start_time>[^"]+)",'
        # Match cookie (e.g., "cookie="JSESSIONID"")
        r'cookie="(?P<cookie>[^"]*)",'
        # Match user (e.g., "user=""")
        r'user="(?P<user>[^"]*)",'
        # Match virtual_server (e.g., "virtual_server="/Common/pool_ppweb4gfoln071 0"")
        r'virtual_server="(?P<virtual_server>[^"]+)",'
        # Match bytes_in (e.g., "bytes_in="68"")
        r'bytes_in="(?P<bytes_in>\d+)",'
        # Match res_start_time (e.g., "res_start_time="2024/05/24 14:33:29"")
        r'res_start_time="(?P<res_start_time>[^"]+)",'
        # Match node (e.g., "node="10.210.56.47%5"")
        r'node="(?P<node>[^"]+)",'
        # Match node_port (e.g., "node_port="80"")
        r'node_port="(?P<node_port>\d+)",'
        # Match http_status (e.g., "http_status="200"")
        r'http_status="(?P<http_status>\d+)",'
        # Match req_elapsed_time (e.g., "req_elapsed_time="193"")
        r'req_elapsed_time="(?P<req_elapsed_time>\d+)",'
        # Match bytes_out (e.g., "bytes_out="41523"")
        r'bytes_out="(?P<bytes_out>\d+)"'
    )
    match = pattern.match(raw_string)
    if match:
        return match.groupdict()
    return {}

# Apply the function to the _raw column and expand the dictionaries into columns
extracted_df = results_df['_raw'].apply(lambda x: pd.Series(extract_variables(x)))

# Concatenate the extracted columns with the original DataFrame
final_df = pd.concat([results_df, extracted_df], axis=1)

#print(final_df.describe())
#print(final_df["http_url"])
# Drop the original _raw column as it is no longer needed
final_df = final_df.drop(columns=['_raw'])

# Display all columns and their values one above the other
#for col in final_df.columns:
#    print(f"Column: {col}")
#    print(final_df[col].to_string(index=False))
#    print("\n" + "="*50 + "\n")

# Optionally, save the final DataFrame to a CSV file
final_df.to_csv('final_results_dataset.csv', index=False)


In [19]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import re

# Charger le modèle et le transformateur TF-IDF déjà entraînés
best_model = grid_search.best_estimator_


# Charger le fichier CSV contenant les nouvelles requêtes
final_result_dataset_path = '/home/onyxia/work/ml-dl-cyber/1-Quick-win/Good_Bad_queries/final_results_dataset.csv'
final_data = pd.read_csv(final_result_dataset_path)


# Vérifier les premières lignes pour s'assurer que les données sont correctement chargées
# print(final_data.head())



# Définir la fonction de tokenisation personnalisée (si non déjà définie dans le contexte)
def custom_tokenizer(text):
    token_pattern = re.compile(r"(?u)\b\w\w+\b|\/|<|>|script|alert|http|\.exe|\.jpg|=")
    return token_pattern.findall(text)

# Utiliser la colonne http_url pour les prédictions
queries_to_predict = final_data['http_url']
#print(queries_to_predict)

In [20]:
# Transformer les nouvelles requêtes en vecteurs TF-IDF
queries_tfidf = best_model.named_steps['vectorizer'].transform(queries_to_predict)

# Effectuer les prédictions
predictions = best_model.named_steps['classifier'].predict(queries_tfidf)
prediction_probas = best_model.named_steps['classifier'].predict_proba(queries_tfidf)

# Ajouter les prédictions et probabilités au DataFrame
final_data['Prediction'] = predictions
final_data['Prediction_Probability_Good'] = prediction_probas[:, 0]
final_data['Prediction_Probability_Bad'] = prediction_probas[:, 1]

# Afficher les résultats pour chaque requête
#for index, row in final_data.iterrows():
#    print(f"Query: {row['http_url']}")
#    print(f"Prediction: {'Bad Query' if row['Prediction'] == 1 else 'Good Query'}")
#    print(f"Prediction Probability - Good: {row['Prediction_Probability_Good']:.4f}")
#    print(f"Prediction Probability - Bad: {row['Prediction_Probability_Bad']:.4f}")
#    print("\n" + "="*50 + "\n")

# Sauvegarder les résultats dans un nouveau fichier CSV
final_data.to_csv('predicted_final_results_dataset.csv', index=False)


In [21]:
import pandas as pd

# Charger le fichier CSV contenant les nouvelles requêtes avec les prédictions
final_result_dataset_path = '/home/onyxia/work/ml-dl-cyber/1-Quick-win/Good_Bad_queries/predicted_final_results_dataset.csv'
final_data = pd.read_csv(final_result_dataset_path)

# Faire un résumé statistique de la colonne 'Prediction'
prediction_summary = final_data['Prediction'].value_counts()
prediction_summary_percentage = final_data['Prediction'].value_counts(normalize=True) * 100

# Créer un DataFrame pour afficher les résultats
summary_df = pd.DataFrame({
    'Count': prediction_summary,
    'Percentage': prediction_summary_percentage
})

# Afficher les résultats
print(summary_df)


            Count  Percentage
Prediction                   
0            3103   99.582798
1              13    0.417202


In [27]:
import pandas as pd

# Charger le fichier CSV contenant les nouvelles requêtes avec les prédictions
final_result_dataset_path = '/home/onyxia/work/ml-dl-cyber/1-Quick-win/Good_Bad_queries/predicted_final_results_dataset.csv'
final_data = pd.read_csv(final_result_dataset_path)

# Filtrer les données pour lesquelles la prédiction est égale à 1
predicted_bad_queries = final_data[final_data['Prediction'] == 1]

# Afficher la colonne 'http_url' des requêtes filtrées
bad_query_urls = predicted_bad_queries['http_url']

print("URLs des requêtes prédites comme mauvaises (Prediction == 1):")
print(bad_query_urls)

URLs des requêtes prédites comme mauvaises (Prediction == 1):
488     "web4gfo.preprod.insee.fr:443/fr/graphique/aja...
974     "web4gfo.preprod.insee.fr:443/fr/graphique/aja...
982     "web4gfo.preprod.insee.fr:443/fr/graphique/aja...
1077    "web4gfo.preprod.insee.fr:443/en/graphique/aja...
1079    "web4gfo.preprod.insee.fr:443/en/graphique/aja...
1085    "web4gfo.preprod.insee.fr:443/en/graphique/aja...
1123    "web4gfo.preprod.insee.fr:443/en/graphique/aja...
1126    "web4gfo.preprod.insee.fr:443/en/graphique/aja...
1146    "web4gfo.preprod.insee.fr:443/en/graphique/aja...
2101    "web4gfo.preprod.insee.fr:443/fr/graphique/aja...
2324    "web4gfo.preprod.insee.fr:443/en/statistiques?...
2325    "web4gfo.preprod.insee.fr:443/langue/en?url=%2...
2340    "web4gfo.preprod.insee.fr:443/fr/statistiques?...
Name: http_url, dtype: object


In [28]:
# Sauvegarder les URLs filtrées dans un nouveau fichier CSV
bad_query_urls.to_csv('bad_query_urls.csv', index=False)


## Itération sur le tokenizer

Les différences entre le tokenizer précédent et le tokenizer amélioré résident principalement dans les motifs supplémentaires et les ajustements qui permettent de capturer une plus grande variété de tokens pertinents. Voici une comparaison détaillée :

### Tokenizer Précédent :
```python
def custom_tokenizer(text):
    # Utilisez une expression régulière pour considérer les mots et certains symboles spéciaux
    token_pattern = re.compile(r"(?u)\b\w\w+\b|\/|<|>|script|alert|http|\.exe|\.jpg|=")
    return token_pattern.findall(text)
```
#### Motifs Capturés :
- `(?u)\b\w\w+\b` : Capturait les mots de 2 caractères ou plus.
- `\/|<|>|script|alert|http|\.exe|\.jpg|=` : Capturait certains symboles spéciaux et sous-chaînes spécifiques comme `script`, `alert`, `http`, `.exe`, `.jpg`.

### Tokenizer Amélioré :
```python
import re

def custom_tokenizer(text):
    # Utilisez une expression régulière améliorée pour capturer les mots, les chiffres, et les symboles spéciaux pertinents
    token_pattern = re.compile(
        r"""
        (?u)\b\w\w+\b           # mots de 2 caractères ou plus
        | \d+                   # séquences de chiffres
        | \bhttps?://\S+\b      # URLs commençant par http ou https
        | [\/<>=&]              # caractères spéciaux courants dans les URL et les requêtes
        | \b\w+\.(com|org|net|fr|in)\b  # domaines courants
        """, re.VERBOSE)
    return token_pattern.findall(text)
```

#### Motifs Capturés :
- `(?u)\b\w\w+\b` : Capturait les mots de 2 caractères ou plus (comme dans la version précédente).
- `\d+` : Ajoute la capture des séquences de chiffres.
- `\bhttps?://\S+\b` : Ajoute la capture des URLs commençant par `http` ou `https`.
- `[\/<>=&]` : Capture les caractères spéciaux courants dans les URLs et les requêtes (maintien de certains symboles du tokenizer précédent).
- `\b\w+\.(com|org|net|fr|in)\b` : Ajoute la capture des domaines courants (comme `.com`, `.org`, `.net`, `.fr`, `.in`).

### Comparaison des Améliorations :
1. **Captures de chiffres** : Le tokenizer amélioré peut maintenant capturer des séquences de chiffres (`\d+`), ce qui est utile pour les dates, les quantités et d'autres informations numériques courantes dans les URLs et les textes.
   
2. **Capture des URLs complètes** : Le motif `\bhttps?://\S+\b` permet de capturer les URLs complètes, contrairement à l'ancien motif qui ne capturait que la sous-chaîne `http`.

3. **Domaines courants** : Le tokenizer amélioré capture les noms de domaines courants comme `.com`, `.org`, `.net`, `.fr`, et `.in`, ajoutant une granularité utile pour l'analyse des URLs.

4. **Expression régulière plus lisible** : L'utilisation de `re.VERBOSE` permet de formater l'expression régulière avec des commentaires et des espaces, rendant le code plus lisible et maintenable.

### Pourquoi ces changements ?
- **Augmentation de la couverture** : Ces ajouts augmentent la couverture des tokens potentiellement pertinents, y compris des éléments spécifiques aux URLs et des chiffres.
- **Réduction des faux positifs et négatifs** : En capturant des motifs plus spécifiques, le tokenizer réduit les chances de marquer incorrectement des requêtes légitimes comme suspectes.
- **Meilleure analyse des données web** : Les nouvelles captures (comme les URLs complètes et les domaines courants) sont particulièrement utiles pour l'analyse de données web, ce qui est pertinent dans le contexte de vos données.

Ces améliorations rendent le tokenizer plus robuste et adaptable à l'analyse des requêtes HTTP, tout en maintenant la simplicité et la lisibilité du code.